# 🧌 RuneLite Goblin Bot

This is a **computer vision bot** that detects and attacks goblins in RuneLite using **YOLOv8** and **Win32 screen capture**.  
The bot tracks goblins in real time, allowing you to manually attack by pressing **'E'**.

## **⚡ Features**
✅ **Real-time goblin tracking** using YOLO  
✅ **Press 'E' to attack**, 'Q' to quit  
✅ **Does NOT require the game window to be active**  
✅ **Works without GPU acceleration (CPU-only)**  

---

## **🚀 Setup & Installation**
### **1️⃣ Install Dependencies**
Make sure you have Python installed, then run:
```bash
pip install -r requirements.txt


In [3]:
import torch
import numpy as np
import mss
import cv2
import time
import pyautogui
import pygetwindow as gw
from PIL import Image
from ultralytics import YOLO

## 📌 get_game_region_auto() – Auto-Detect RuneLite Window
This function automatically detects the RuneLite window and extracts its position and size.
It removes the need for manual mouse placement, making setup faster and more efficient.

### 🛠️ How It Works
##### 1️⃣ Finds RuneLite's window using getWindowsWithTitle("RuneLite")
##### 2️⃣ Extracts window position & size (top, left, width, height)
##### 3️⃣ Returns the game region as a dictionary
##### 4️⃣ Prints confirmation when the window is found

In [4]:
def get_game_region_auto():
    # Find the RuneLite window
    windows = gw.getWindowsWithTitle("RuneLite")

    if not windows:
        print("❌ RuneLite window not found! Make sure it's open.")
        return None

    runelite_window = windows[0]  # Assuming the first match is the correct window

    # Extract window coordinates
    game_region = {
        "top": runelite_window.top,
        "left": runelite_window.left,
        "width": runelite_window.width,
        "height": runelite_window.height
    }

    print("\n🎯 **Game Window Detected Automatically!**")
    print(f"📍 **Game Region:** Top={game_region['top']}, Left={game_region['left']}, Width={game_region['width']}, Height={game_region['height']}\n")

    return game_region

# ✅ Automatically detect RuneLite window
game_region = get_game_region_auto()



🎯 **Game Window Detected Automatically!**
📍 **Game Region:** Top=101, Left=947, Width=836, Height=582



# 🧌
####  📌 1. capture_runeLite_window(window_name)
##### Captures the RuneLite game window using the Win32 API.
##### Returns a screenshot as a NumPy array.

#### 📌 2. delayed_bot_and_tracking(model)
##### Opens an OpenCV window for real-time detection.
##### Captures RuneLite’s window every frame.
##### Runs YOLO object detection to find goblins.
##### Tracks the closest goblin to the center of the screen.




In [ ]:
import time
import cv2
import numpy as np
import pyautogui
import keyboard  
import win32gui, win32ui, win32con
from ultralytics import YOLO
from collections import deque

# **RuneLite Window Title**
window_name = "RuneLite - SavignonRed"

# **Goblin Class ID = 2
TARGET_CLASSES = {2}

def capture_runeLite_window(window_name):
    """Captures the RuneLite window as an image using Win32 API."""
    hwnd = win32gui.FindWindow(None, window_name)
    if not hwnd:
        print("❌ Error: RuneLite window not found.")
        return None, None

    left, top, right, bottom = win32gui.GetWindowRect(hwnd)
    width, height = right - left, bottom - top

    hdc = win32gui.GetWindowDC(hwnd)
    dc_obj = win32ui.CreateDCFromHandle(hdc)
    mem_dc = dc_obj.CreateCompatibleDC()
    bmp = win32ui.CreateBitmap()
    bmp.CreateCompatibleBitmap(dc_obj, width, height)
    mem_dc.SelectObject(bmp)
    mem_dc.BitBlt((0, 0), (width, height), dc_obj, (0, 0), win32con.SRCCOPY)

    bmp_info = bmp.GetInfo()
    img = np.frombuffer(bmp.GetBitmapBits(True), dtype=np.uint8)
    img.shape = (bmp_info['bmHeight'], bmp_info['bmWidth'], 4)

    win32gui.DeleteObject(bmp.GetHandle())
    mem_dc.DeleteDC()
    dc_obj.DeleteDC()
    win32gui.ReleaseDC(hwnd, hdc)

    return cv2.cvtColor(img, cv2.COLOR_BGRA2RGB), (left, top, width, height)

def delayed_bot_and_tracking(model):
    """Tracks goblins but only clicks when 'E' is pressed."""
    
    print("✅ Tracking started. Press 'E' to attack. Press 'Q' to exit.")

    cv2.namedWindow("RuneLite - Goblin Tracking", cv2.WINDOW_NORMAL)  # Create OpenCV window
    track_history = deque(maxlen=5)
    last_goblin_position = None  # Stores last detected goblin position

    while True:
        frame, game_region = capture_runeLite_window(window_name)
        if frame is None:
            print("❌ Error: Could not capture RuneLite window.")
            break

        pil_img = Image.fromarray(frame)
        results = model(pil_img, imgsz=320)  # Faster inference (no CUDA)

        if not results[0].boxes or len(results[0].boxes) == 0:
            print("⚠️ No goblins detected.")
            last_goblin_position = None
            continue

        boxes = results[0].boxes.xyxy.cpu().numpy()
        classes = results[0].boxes.cls.cpu().numpy()
        confidences = results[0].boxes.conf.cpu().numpy()

        img_with_boxes = np.array(results[0].plot())

        # **Fix OpenCV Window Display Issues**
        cv2.imshow("RuneLite - Goblin Tracking", img_with_boxes)
        if cv2.getWindowProperty("RuneLite - Goblin Tracking", cv2.WND_PROP_VISIBLE) < 1:
            print("❌ OpenCV window closed. Exiting bot.")
            break

        cv2.waitKey(1)  # Keep OpenCV window responsive

        # **Global Key Listener (E = Attack, Q = Quit)**
        if keyboard.is_pressed('q'):
            print("❌ Stopping bot & tracking.")
            break

        # **Find the best goblin to attack (closest to screen center)**
        best_box, best_center = None, None
        best_distance = float("inf")

        screen_center_x = game_region[2] // 2
        screen_center_y = game_region[3] // 2

        for i, box in enumerate(boxes):
            if classes[i] not in TARGET_CLASSES or confidences[i] < 0.3:
                continue  # Only select goblins with confidence > 0.3

            x1, y1, x2, y2 = map(int, box)
            box_center_x = (x1 + x2) // 2
            box_center_y = (y1 + y2) // 2

            distance = ((box_center_x - screen_center_x) ** 2 + (box_center_y - screen_center_y) ** 2) ** 0.5

            if distance < best_distance:
                best_distance = distance
                best_box = box
                best_center = (box_center_x, box_center_y)

        if best_box is not None:
            # Convert to screen coordinates
            screen_x = game_region[0] + best_center[0]
            screen_y = game_region[1] + best_center[1]

            last_goblin_position = (screen_x, screen_y)  # Store last detected goblin

        # **Click only when 'E' is pressed (from ANY window)**
        if keyboard.is_pressed('e') and last_goblin_position:
            print(f"🎯 Attacking goblin at ({last_goblin_position[0]}, {last_goblin_position[1]})...")
            pyautogui.moveTo(last_goblin_position[0], last_goblin_position[1], duration=np.random.uniform(0.1, 0.2))
            pyautogui.click()
            print("✅ Clicked on goblin.")

    cv2.destroyAllWindows()
    print("✅ Bot & tracking loop exited safely.")

# Load YOLOv11 model (No CUDA)
model = YOLO(r"C:/Users/nicol/Computer Vision Group/best.pt")  

# Run bot
delayed_bot_and_tracking(model)


✅ Tracking started. Press 'E' to attack. Press 'Q' to exit.

0: 224x320 5 goblins, 54.6ms
Speed: 2.6ms preprocess, 54.6ms inference, 0.8ms postprocess per image at shape (1, 3, 224, 320)

0: 224x320 4 goblins, 53.9ms
Speed: 1.1ms preprocess, 53.9ms inference, 0.7ms postprocess per image at shape (1, 3, 224, 320)

0: 224x320 4 goblins, 44.1ms
Speed: 1.0ms preprocess, 44.1ms inference, 0.6ms postprocess per image at shape (1, 3, 224, 320)

0: 224x320 5 goblins, 49.5ms
Speed: 1.0ms preprocess, 49.5ms inference, 0.6ms postprocess per image at shape (1, 3, 224, 320)

0: 224x320 5 goblins, 51.3ms
Speed: 2.5ms preprocess, 51.3ms inference, 0.7ms postprocess per image at shape (1, 3, 224, 320)

0: 224x320 5 goblins, 43.7ms
Speed: 1.1ms preprocess, 43.7ms inference, 0.6ms postprocess per image at shape (1, 3, 224, 320)

0: 224x320 5 goblins, 43.5ms
Speed: 0.9ms preprocess, 43.5ms inference, 1.0ms postprocess per image at shape (1, 3, 224, 320)

0: 224x320 5 goblins, 43.7ms
Speed: 0.8ms preproce

# 👾 Happy Botting! 🧌

For any issues check read.me file